# Joinning and Pivoting DataFrames

In [0]:
from pyspark.sql.functions import *

In [0]:
path_countries = "dbfs:/FileStore/sample_data/countries.txt"
path_regions = 'dbfs:/FileStore/sample_data/country_regions.csv'

In [0]:
countries_df = (spark.read
    .format('csv')
    .option('header','true')
    .option('inferschema','true')
     .option('sep', '\t')
    .load(path_countries)
)

In [0]:
path_regions = 'dbfs:/FileStore/sample_data/country_regions.csv'

regions_df = (spark.read
    .format('csv')
    .option('header','true')
    .option('inferschema','true')
     .option('sep', ',')
    .load(path_regions)
)

In [0]:
display(regions_df.limit(10))

ID,NAME
10,America
20,Europe
30,Asia
40,Oceania
50,Africa



When piviting a DF, you apply the pivot method after the groupby method but before the aggregation. 

In [0]:
countries_df.groupBy('region_id','sub_region_id') \
    .agg(sum(col('population')) \
    .alias('population')) \
    .orderBy('population', ascending = False) \
    .limit(10) \
    .display()

region_id,sub_region_id,population
30,30,3836422762
30,60,3345222196
50,160,2132574302
30,100,1324023612
10,10,1296241438
10,80,733201928
20,140,586889826
30,170,550649626
50,40,483561536
20,150,391044820


In [0]:
countries_df \
    .groupBy('sub_region_id') \
    .pivot('region_id') \
    .agg(sum(col('population'))) \
    .limit(10) \
    .display()


    # Pivots are good for creating matrices

sub_region_id,10,20,30,40,50
140,null,586889826,null,null,null
null,null,null,null,2212,null
40,null,null,null,null,483561536
20,null,null,null,1359678,null
120,null,211583456,null,null,null
100,null,null,1324023612,null,null
130,null,null,null,21837034,null
10,1296241438,null,null,null,null
50,null,null,null,1087572,null
80,733201928,null,null,null,null


In [0]:
population_df = countries_df \
    .join(regions_df, countries_df['region_id'] == regions_df['id'], how='left') \
    .select(
        countries_df['name'].alias('country_name'),
        regions_df['name'].alias('region_name'),
        countries_df['population']
    )
 
display(population_df.head(10))

country_name,region_name,population
Afghanistan,Asia,38041754
Albania,Europe,2880917
Algeria,Africa,43053054
American Samoa,Oceania,55312
Andorra,Europe,77142
Angola,Africa,31825295
Anguilla,America,14869
Antarctica,Oceania,1106
Antigua and Barbuda,America,97118
Argentina,America,44780677


In [0]:
population_df \
    .groupBy('region_name') \
    .agg(sum('population').alias('population')) \
    .orderBy('population', ascending=False) \
    .display()  


region_name,population
Asia,4601371198
Africa,1308067919
America,1014721683
Europe,747205974
Oceania,42133644


In [0]:
 population_df \
    .groupBy('country_name') \
    .pivot('region_name') \
    .sum('population') \
    .orderBy('country_name') \
    .limit(10) \
    .display()



country_name,Africa,America,Asia,Europe,Oceania
Afghanistan,null,null,38041754,null,null
Albania,null,null,null,2880917,null
Algeria,43053054,null,null,null,null
American Samoa,null,null,null,null,55312
Andorra,null,null,null,77142,null
Angola,31825295,null,null,null,null
Anguilla,null,14869,null,null,null
Antarctica,null,null,null,null,1106
Antigua and Barbuda,null,97118,null,null,null
Argentina,null,44780677,null,null,null


In [0]:
population_df \
    .groupBy('region_name','country_name') \
    .agg(sum('population').alias('population')) \
    .orderBy('population', ascending=False) \
    .limit(10) \
    .display()  


region_name,country_name,population
Asia,China,1433783686
Asia,India,1366417754
America,United States of America,329064917
Asia,Indonesia,270625568
Asia,Pakistan,216565318
America,Brazil,211049527
Africa,Nigeria,200963599
Asia,Bangladesh,163046161
Europe,Russian Federation,145872256
America,Mexico,127575529


In [0]:
# Use Union to append two dataframes. Must have the same number of columns.

---------------------------------------------------------------------------
ParseException                            Traceback (most recent call last)
File <command-1657774773984061>, line 4
      1 # Unpivot
      3 population_df \
----> 4     .select('country_name', expr(""))

File /databricks/spark/python/pyspark/sql/utils.py:264, in try_remote_functions.<locals>.wrapped(*args, **kwargs)
    262     return getattr(functions, f.__name__)(*args, **kwargs)
    263 else:
--> 264     return f(*args, **kwargs)

File /databricks/spark/python/pyspark/sql/functions/builtin.py:6590, in expr(str)
   6560 @_try_remote_functions
   6561 def expr(str: str) -> Column:
   6562     """Parses the expression string into the column that it represents
   6563 
   6564     .. versionadded:: 1.5.0
   (...)
   6588     +-----+------------+
   6589     """
-> 6590     return _invoke_function("expr", str)

File /databricks/spark/python/pyspark/sql/functions/builtin.py:103, in _invoke_function(name, *args)
 

In [0]:
countries_df.count()

249

In [0]:
countries_df = countries_df.union(countries_df)

In [0]:
countries_df.count()

498

In [0]:
countries_df.printSchema()

root
 |-- COUNTRY_ID: integer (nullable = true)
 |-- NAME: string (nullable = true)
 |-- NATIONALITY: string (nullable = true)
 |-- COUNTRY_CODE: string (nullable = true)
 |-- ISO_ALPHA2: string (nullable = true)
 |-- CAPITAL: string (nullable = true)
 |-- POPULATION: integer (nullable = true)
 |-- AREA_KM2: double (nullable = true)
 |-- REGION_ID: integer (nullable = true)
 |-- SUB_REGION_ID: integer (nullable = true)
 |-- INTERMEDIATE_REGION_ID: integer (nullable = true)
 |-- ORGANIZATION_REGION_ID: integer (nullable = true)

